In [ ]:
import logging
import re
from urllib.parse import urljoin,urlencode
import multiprocessing
import time
import json
from os import makedirs
from os.path import exists
import pandas as pd
from lxml import etree

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By

In [ ]:
logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s: %(message)s')
# 基础网址和爬取页码数量设定
BASE_URL='https://www.9fzt.com/marketCenter/aStockMarket.html?tab=0'
TOT_PAGEs=10
def get_browser():
    option=ChromeOptions()
    option.add_experimental_option('excludeSwitches',['enable-automation'])
    option.add_experimental_option('useAutomationExtension',False)
    # 设置不显式地显示浏览器
    option.add_argument('--headless')
    browser=webdriver.Chrome(options=option)
    browser.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument",
                            {'source': 'Object.defineProperty(navigator,"webdriver",{get:()=>undefind})'})
    browser.implicitly_wait(10)
    return browser


# %%
# 解析 html 中的 urls
def parse_page(page_html,protol_type='https:'):
    pattern=re.compile('序号(.*)',re.S)
    stock_urls=re.findall(pattern,page_html)
    pattern=re.compile('<a href="(.*?)".*? class="bluelink ff_din-medium fw-500" target="_blank" rel="noopener">',re.S)
    stock_urls=re.findall(pattern,str(stock_urls))
    stock_urls=[urljoin(protol_type,url) for url in stock_urls]
    return stock_urls


# %%
# 爬取股票数据列表 urls
def scrape_stock_list(browser,page_num):
    browser.get(BASE_URL)
    urls=[]
    WebDriverWait(browser,20,0.5).until(lambda browser:len(
        browser.find_element(By.XPATH,'//*[@id="__next"]/div/div[3]/div[2]/ul[20]/li[13]/span').text)>0)
    for page in range(page_num):
        if(page >= 4):
            # continue
            urls+=(parse_page(browser.page_source))
        time.sleep(0.2)
        ac=ActionChains(browser)
        # 鼠标移动到下一页按钮上
        ac.move_to_element(browser.find_element(By.NAME, 'whj_nextPage')).perform()
        # 点击确定跳转至下一页
        ac.click(browser.find_element(By.NAME, 'whj_nextPage')).perform()
        time.sleep(0.2)
        WebDriverWait(browser,10,0.5).until(lambda browser: len(
            browser.find_element(By.XPATH,'//*[@id="__next"]/div/div[3]/div[2]/ul[20]/li[13]/span').text)>0)
    print(urls)
    return urls


browser = webdriver.Chrome()
urls=scrape_stock_list(browser,TOT_PAGEs)



In [ ]:
# %%
# 爬取公司各个页面 html ，以list形式返回
def scrape_company_list(browser,urls):
    companys=[]
    cnt = 0
    for url in urls:
        logging.info('scraping %s...',url)
        browser.get(url)

        logging.info('scraping 公司资料...')
        browser.find_element(By.LINK_TEXT, '公司资料').click()
        browser.switch_to.window(browser.window_handles[1])
        WebDriverWait(browser,15,2).until(lambda browser: len(
            browser.find_element(By.XPATH,'//*[@id="xxqk"]/div[2]/div/div/section/div/ul/li[4]/div[2]/section/div/div').text)>0)
        data=browser.page_source
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

        logging.info('scraping 股东研究...')
        browser.find_element(By.LINK_TEXT, '股东研究').click()
        browser.switch_to.window(browser.window_handles[1])
        WebDriverWait(browser,10,2).until(lambda browser: len(browser.find_element(By.XPATH, '//*[@id="sdgd"]/div[2]/div/div/section/div[2]/div/div/div/div/div/table').text)>0)
        shareholder=browser.page_source
        browser.close()
        browser.switch_to.window(browser.window_handles[0])


        logging.info('scraping 涉及概念...')
        browser.find_element(By.LINK_TEXT, '最新动态').click()
        browser.switch_to.window(browser.window_handles[1])
        WebDriverWait(browser,10,2).until(lambda browser: len(browser.find_element(By.XPATH, '//*[@id="gsgy"]/div[2]/div/div/section/div/ul/li[2]/div[2]/div/span').text)>0)
        concept=browser.page_source
        browser.close()
        browser.switch_to.window(browser.window_handles[0])


        logging.info('scraping 公司公告...')
        browser.find_element(By.LINK_TEXT, '更多').click()
        browser.switch_to.window(browser.window_handles[1])
        WebDriverWait(browser,10,0.5).until(lambda browser: len(
            browser.find_element(By.XPATH,'//*[@id="app"]/div/section/div[1]/section/div[1]/div/div/div[2]/ul/li[12]/div').text)>0)
        # 模拟移动点击跳转
        time.sleep(0.3)
        ac = ActionChains(browser)
        ac.move_to_element(browser.find_element(By.XPATH,'//*[@id="app"]/div/section/div[1]/section/div[1]/div/div/div[2]/ul/li[12]/div')).perform()
        ac.click(browser.find_element(By.XPATH,'//*[@id="app"]/div/section/div[1]/section/div[1]/div/div/div[2]/ul/li[12]/div')).perform()
        time.sleep(0.3)
        # 等待公告加载完成
        WebDriverWait(browser, 10, 2).until(lambda browser: len(
            browser.find_element(By.XPATH,
                                 '//*[@id="MediaGszx0"]').text)>0)
        news=browser.page_source
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

        companys.append([data, shareholder, concept, news])
        print(cnt)

        cnt += 1

        # if(cnt == 20):
        #     break
    return companys


companys=scrape_company_list(browser,urls)

In [ ]:
# %%
# 使用xpath匹配
stock_code_name_xpath='//*[@id="app"]/div/section/div[1]/section/div[1]/div/div/div[1]/h3/i/text()'
company_name_xpath='//*[@id="xxqk"]/div[2]/div/div/section/div/ul/li[1]/div[2]/div/span/text()'
company_time_xpath='//*[@id="xxqk"]/div[2]/div/div/section/div/ul/li[5]/div[2]/div/span/text()'
company_business_xpath='//*[@id="xxqk"]/div[2]/div/div/section/div/ul/li[23]/div[2]/div/span/text()'
company_representative_xpath='//*[@id="xxqk"]/div[2]/div/div/section/div/ul/li[9]/div[2]/div/span/text()'
company_manager_xpath='//*[@id="xxqk"]/div[2]/div/div/section/div/ul/li[10]/div[2]/div/span/text()'
company_hangye_xpath='//*[@id="xxqk"]/div[2]/div/div/section/div/ul/li[8]/div[2]/div/span/text()'
company_secretary_xpath='//*[@id="xxqk"]/div[2]/div/div/section/div/ul/li[11]/div[2]/div/span/text()'
company_phone_xpath='//*[@id="xxqk"]/div[2]/div/div/section/div/ul/li[14]/div[2]/div/span/text()'
company_place_pattern='//*[@id="xxqk"]/div[2]/div/div/section/div/ul/li[18]/div[2]/div/span/text()'

# 使用re匹配
city_pattern=re.compile('(.*?省)?(.*?市)?',re.S)
company_id_pattern=re.compile('(</html>.*)', re.S)

# 对已经得到的 html 进行解析——股票基本数据
def parse_company_data(html):
    html=etree.HTML(html)
    code_name = html.xpath(stock_code_name_xpath)[0].split(' ')
    stock_code = code_name[0]
    stock_name = code_name[1]
    company_place=html.xpath(company_place_pattern)[0]
    province='--'
    city='--'
    if re.search(city_pattern,company_place).group(1):
        province=re.search(city_pattern,company_place).group(1)
    if re.search(city_pattern,company_place).group(2):
        city=re.search(city_pattern,company_place).group(2)
    company_name=html.xpath(company_name_xpath)[0]
    company_time=html.xpath(company_time_xpath)[0]
    company_hangye=html.xpath(company_hangye_xpath)[0]
    company_business=html.xpath(company_business_xpath)[0]
    company_representative=html.xpath(company_representative_xpath)[0]
    company_manager=html.xpath(company_manager_xpath)[0]
    company_secretary=html.xpath(company_secretary_xpath)[0]
    company_phone=html.xpath(company_phone_xpath)[0]
    return [stock_code,stock_name,company_name,province,city,company_time,company_hangye,company_business,company_representative,
            company_manager,company_secretary,company_phone]

def get_table_company_data(data):
    table=pd.DataFrame(columns=['stock_code','stock_name','company_name','province','city',
                                'company_time','行业','company_business','company_representative',
                                'company_manager','company_secretary','company_phone'])
    for item in data:
        table.loc[len(table)]=parse_company_data(item)
    return table

data=[company[0] for company in companys]
company_data_table=get_table_company_data(data)
company_data_table.to_csv('company_data_table.csv',index=False,encoding='gbk')

In [ ]:
# 对已经得到的 html 进行解析——股东基本信息
def parse_sharehold(html, id):
    company_id = company_id_pattern.search(html).group(1)[7:]
    html = etree.HTML(html)
    subfix0 = '"]/text()'
    subfix1 = ']/span/text()'
    name0 = '//*[@id="SHNameTopTenSH'
    nature0 = '//*[@id="SHNatureTopTenSH'
    type0 = '//*[@id="ShareTypeTopTenSH'
    prefix0 = '//*[@id="sdgd"]/div[2]/div/div/section/div[2]/div/div/div/div/div/table/tbody/tr['
    number1 = ']/td[5]/span/text()'
    rate1 = ']/td[6]/span/text()'
    change1 = ']/td[7]/span/text()'
    name = html.xpath(name0 + str(id) + subfix0)[0]
    nature = html.xpath(nature0 + str(id) + subfix0)[0]
    type = html.xpath(type0 + str(id) + subfix0)[0]
    number = html.xpath(prefix0 + str(id + 1) + number1)[0]
    rate = html.xpath(prefix0 + str(id + 1) + rate1)[0]
    change = html.xpath(prefix0 + str(id + 1) + change1)[0]
    return [company_id, name,  nature, type, number, rate, change]


def get_table_sharehold_data(data):
    table = pd.DataFrame(columns=['stock_code', '股东名称', '股东性质', '股份类型', '持股数(股)', '占总股本比例', '增减(股)'])
    # , '股东性质'
    for item in data:
        for id in range(10):
            table.loc[len(table)] = parse_sharehold(item, id)

    return table


sharehold = [company[1] for company in companys]
for i in range(len(sharehold)):
    sharehold[i] = sharehold[i] + company_data_table.loc[i][0]
share_data_table = get_table_sharehold_data(sharehold)

# print(company_data_table)
share_data_table.to_csv('sharehold_data_table.csv', index=False, encoding='gbk')

In [ ]:
# 对已经得到的 html 进行解析——基金基本信息
def parse_fund(html, id):
    company_id = company_id_pattern.search(html).group(1)[7:]
    html = etree.HTML(html)
    fund_id_xpath0 = '//*[@id="jjcg"]/div[2]/div/div/section/div[2]/div/div/div/div/div/table/tbody/tr['
    fund_id_xpath1 = ']/td[2]/text()'
    fund_name_xpath0 = '//*[@id="FundNameFundHold'
    fund_name_xpath1 = '"]/text()'
    fund_number_xpath0 = '//*[@id="jjcg"]/div[2]/div/div/section/div[2]/div/div/div/div/div/table/tbody/tr['
    fund_number_xpath1 = ']/td[4]/span/text()'
    fund_rate_xpath0 = '//*[@id="jjcg"]/div[2]/div/div/section/div[2]/div/div/div/div/div/table/tbody/tr['
    fund_rate_xpath1 = ']/td[6]/span/text()'
    if len(html.xpath(fund_number_xpath0 + str(id + 1) + fund_number_xpath1)) == 0:
        return None
    fund_id = html.xpath(fund_id_xpath0 + str(id + 1) + fund_id_xpath1)[0]
    fund_name = html.xpath(fund_name_xpath0 + str(id) + fund_name_xpath1)[0]
    fund_number = html.xpath(fund_number_xpath0 + str(id + 1) + fund_number_xpath1)[0]
    fund_rate = html.xpath(fund_rate_xpath0 + str(id + 1) + fund_rate_xpath1)[0]
    return [company_id, fund_id, fund_name, fund_number, fund_rate]
def get_table_fund_data(data):
    table = pd.DataFrame(
        columns=['stock_code', '基金代码', '基金名称', '持股数(股)', '占总股本比例'])
    for item in data:
        for id in range(10):
            tmp = parse_fund(item, id)
            if tmp != None:
                table.loc[len(table)] = parse_fund(item, id)
    return table
fund_data_table = get_table_fund_data(sharehold)
fund_data_table.to_csv('fund_data_table.csv', index=False,encoding='gbk')
# print(fund_data_table)


In [ ]:
# 对已经得到的 html 进行解析——股票概念基本信息
def get_table_cept_data(data):
    table = pd.DataFrame(columns=['stock_code', '概念'])

    for item in data:
        company_id = company_id_pattern.search(item).group(1)[7:]
        html = etree.HTML(item)
        target = html.xpath('//*[@id="gsgy"]/div[2]/div/div/section/div/ul/li[2]/div[2]/div/span/text()')[0]
        op = target.split(',')
        for x in op:
            table.loc[len(table)] = [company_id, x]
    return table
concept = [company[2] for company in companys]
for i in range(len(concept)):
    concept[i] = concept[i] + company_data_table.loc[i][0]
fund_data_table = get_table_cept_data(concept)
fund_data_table.to_csv('concept_data_table.csv', index=False,encoding='gbk')
print(fund_data_table)


In [ ]:
# 对已经得到的 html 进行解析——公告基本信息
def parse_news(html, id):
    company_id = company_id_pattern.search(html).group(1)[7:]
    html = etree.HTML(html)
    news_date_xpath0 = '//*[@id="gsggDetail"]/div[2]/section/div/div/section/div/div/div/div/div/div/table/tbody/tr['
    news_date_xpath1 = ']/td[1]/text()'
    news_name_xpath0 = '//*[@id="ggTitleGszxDetail'
    news_name_xpath1 = '"]/text()'
    if len(html.xpath(news_name_xpath0 + str(id) + news_name_xpath1)) == 0:
        return None
    news_date = html.xpath(news_date_xpath0 + str(id+1) + news_date_xpath1)[0]
    news_name = html.xpath(news_name_xpath0 + str(id) + news_name_xpath1)[0]
    print(company_id)
    return [company_id, news_date, news_name]

def get_table_news_data(data):
    table = pd.DataFrame(columns=['stock_code', '时间', '公告'])

    for item in data:
        for id in range(10):
            tmp = parse_news(item, id)
            if tmp != None:
                table.loc[len(table)] = parse_news(item, id)

    return table

news = [company[3] for company in companys]
for i in range(len(news)):
    news[i] = news[i] + company_data_table.loc[i][0]
news_data_table = get_table_news_data(news)
news_data_table.to_csv('news_data_table.csv', index=False,encoding='gbk')
print(news_data_table)